In [1]:
import spacy
import numpy as np
from scipy.stats import pearsonr, spearmanr
import torch

In [2]:
from TRExData.LamaTRExData import LamaTRExData
from TRExData.HardTRExData import HardKnownTRExData
from TypologyQuerier import TypologyQuerier 
from SentenceComparison.SentenceComparison import SentenceComparison

In [3]:
from relation_templates.templates import get_templates, get_relation_meta, relations, relation_names, get_relation_cardinality, KEYS
from ModelHelpers.data_helpers import create_known_filter_getter, create_hard_filter_getter
from ModelHelpers.fill_mask_helpers import get_probability_from_pipeline_for_token
from PipelineCacheWrapper.PipelineCacheWrapper import PipelineCacheWrapper
from transformers import logging

In [4]:
logging.set_verbosity_error()

In [5]:
MASK = "[MASK]"
TOP_K = 10

In [6]:
nlp = spacy.load("en_core_web_sm")

In [7]:
unmasker = PipelineCacheWrapper('fill-mask', model='bert-base-cased', top_k=TOP_K)

TREx = LamaTRExData(relations = relations)
TREx.load()

In [8]:
def model_to_tokens(sentence):
    result = unmasker(sentence)
    result = sorted(result, key=lambda entry: entry["score"], reverse=True)
    return [entry["token_str"] for entry in result]

get_hard_filter = create_hard_filter_getter(model_to_tokens, get_templates)
get_known_filter = create_known_filter_getter(model_to_tokens, get_templates, get_relation_cardinality)
TREx = HardKnownTRExData(get_hard_filter, get_known_filter, relations = relations)
TREx.load()
relations = list(TREx.data.keys())

In [9]:
similarities = {}
for rel in relations:
    sims = []
    templates = get_templates(rel, "subject", "object")
    simple_sentence = nlp(templates[KEYS[0]])
    for key in KEYS[1:]:
        sentence = nlp(templates[key])
        sims.append(simple_sentence.similarity(sentence))
        
    similarities[rel] = sims
    

/var/folders/_c/ldv842y12352htyjfychr6yw0000gr/T/ipykernel_15016/1753012268.py:8: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  sims.append(simple_sentence.similarity(sentence))


In [10]:
querier = TypologyQuerier(unmasker, relations, TOP_K, MASK)
querier.query(TREx.data)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 738/738 [00:00<00:00, 26296.13it/s]


In [11]:
unmasker.save_to_cache()

In [12]:
accuracies = querier.results_for_calculation()
np_accuracies = np.array(accuracies)[1:,2:].flatten().astype(float)

sims_for_calc = [[rel[0]] + similarities[rel[0]] for rel in accuracies[1:]]
np_similarities = np.array(sims_for_calc)[:,1:].flatten().astype(float)

In [13]:
r_corr , _ = pearsonr(np_accuracies, np_similarities)
r_corr

0.1950502303402183

In [14]:
r_corr , _ = spearmanr(np_accuracies, np_similarities)
r_corr

0.1454963648040231

In [15]:
unmasker = PipelineCacheWrapper('fill-mask', model='bert-base-cased', top_k=1000)

In [16]:
@torch.no_grad()
def metric(sentence: str, token: str):
    prob = get_probability_from_pipeline_for_token(unmasker(sentence), token)
    return prob

In [17]:
Comparer = SentenceComparison(relations, get_templates, metric, MASK, get_relation_meta)
Comparer.compare(TREx.data)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 738/738 [00:00<00:00, 5861.94it/s]


In [18]:
probabilities = Comparer.make_global_comparison()
np_probabilities = np.array(probabilities)[:,2:].flatten().astype(float)

sims_for_calc = [[rel[0]] + similarities[rel[0]] for rel in probabilities]
np_similarities = np.array(sims_for_calc)[:,1:].flatten().astype(float)

In [19]:
r_corr , _ = pearsonr(np_probabilities, np_similarities)
r_corr

0.16128386890278262

In [20]:
r_corr , _ = spearmanr(np_probabilities, np_similarities)
r_corr

0.21673483165987997

In [21]:
unmasker.save_to_cache()

In [ ]:
import math

def kll_metric(left_dist, right_dist):
    value = 0.0
    for p, q in zip(left_dist, right_dist):
        if p > 0 and q > 0:
            value += p * math.log(p/q)
    return round(value, 3)

In [ ]:
Comparer.plot_heat_map(kll_metric, "KLL Divergence for sentences type distributions")